<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/LS_DS_241_Begin_modeling_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science — Model Validation_ 

# Begin the modeling process

Objectives
- Train/Validate/Test split
- Cross-Validation
- Begin with baselines

## Get the Bank Marketing dataset

You have several ways you can get the dataset:

#### Kaggle
- Download from the [Kaggle competition page](https://www.kaggle.com/c/ds2-model-validation/data)
- Use the Kaggle API

#### GitHub
- Clone the [repo](https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/tree/master/module-1-begin-modeling-process/bank-marketing)
- Download from the repo:

In [2]:
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_features.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_labels.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/test_features.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/sample_submission.csv   

--2019-03-04 22:26:12--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3833210 (3.7M) [text/plain]
Saving to: ‘train_features.csv.1’

train_features.csv. 100%[===================>]   3.66M  --.-KB/s    in 0.1s    

2019-03-04 22:26:13 (35.0 MB/s) - ‘train_features.csv.1’ saved [3833210/3833210]

--2019-03-04 22:26:14--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.gi

In [3]:
%cd bank-marketing

[Errno 2] No such file or directory: 'bank-marketing'
/content


In [0]:
content/test_features.csv

In [6]:
import pandas as pd

train_features = pd.read_csv('/content/train_features.csv.1')
train_labels = pd.read_csv('/content/train_labels.csv.1')
test_features = pd.read_csv('/content/test_features.csv.1')
sample_submission = pd.read_csv('/content/sample_submission.csv.1')

train_features.shape, train_labels.shape, test_features.shape, sample_submission.shape

((30891, 20), (30891, 2), (10297, 20), (10297, 2))

In [0]:
y_train = train_labels['y']
X_train = train_features.drop(columns='id')
X_test  = test_features.drop(columns='id')

## Train/Validation/Test split

How can we get from a two-way split, to a three-way split?

We can use the [**`sklearn.model_selection.train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split the training data into training and validation data.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23168, 19), (7723, 19), (23168,), (7723,))

Fit on the training set.

Predict and score with the validation set.

## Majority class baseline

Determine the majority class:

In [10]:
y_train.value_counts(normalize=True)

0    0.886266
1    0.113734
Name: y, dtype: float64

Guess the majority class for every prediction:

In [0]:
majority_class = 0
y_pred = [majority_class] * len(y_val)

#### [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

Baseline accuracy by guessing the majority class for every prediction:

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.8905865596270879

#### [`sklearn.metrics.roc_auc_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

Baseline "ROC AUC" score by guessing the majority class for every prediction:

In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

0.5

## Fast first models

### Ignore rows/columns with nulls

This dataset doesn't have any nulls:

In [0]:
X_train.isnull().sum()

### Ignore nonnumeric features

Here are the numeric features:

In [16]:
import numpy as np
X_train.describe(include=np.number)

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000
mean,39.964952,2.558745,962.093103,0.175199,0.083348,93.575132,-40.493918,3.621091,5166.979118
std,10.372040,2.783834,187.862101,0.501901,1.572407,0.577146,4.632871,1.735941,72.352461
min,17.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,56.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


Here are the nonnumeric features:

In [17]:
X_train.describe(exclude=np.number)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome
count,23168,23168,23168,23168,23168,23168,23168,23168,23168,23168
unique,12,4,8,3,3,3,2,10,5,3
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent
freq,5951,14003,6826,18265,12137,19088,14741,7773,4813,19989


Just select the nonnumeric features:

In [0]:
X_train_numeric = X_train.select_dtypes(np.number)
X_val_numeric = X_val.select_dtypes(np.number)

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_numeric, y_train)

y_pred = model.predict(X_val_numeric)
roc_auc_score(y_val, y_pred)

0.5968521535949456

### With Scaler

In [20]:
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_numeric)
X_val_scaled = scaler.transform(X_val_numeric)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_numeric, y_train)

y_pred = model.predict(X_val_scaled)
roc_auc_score(y_val, y_pred)

0.5516752668227828

### Same, as a pipeline

In [21]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    StandardScaler(), 
    LogisticRegression(solver='lbfgs')
)

pipeline.fit(X_train_numeric, y_train)

y_pred = pipeline.predict(X_val_numeric)
roc_auc_score(y_val, y_pred)

0.5959595038464119

### Encode "low cardinality" categoricals

One-hot encode the "low cardinality" categoricals

In [22]:
X_train.select_dtypes(exclude=np.number).nunique()

job            12
marital         4
education       8
default         3
housing         3
loan            3
contact         2
month          10
day_of_week     5
poutcome        3
dtype: int64

Install the Category Encoder library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [24]:
!pip install category_encoders


    100% |████████████████████████████████| 61kB 1.8MB/s 


In [65]:
import category_encoders as ce

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)
roc_auc_score(y_val, y_pred)

0.6186510114575071

In [0]:
y_test_pred = pipeline.predict(X_test)

In [0]:
y_test_pred=pd.DataFrame(y_test_pred)

In [0]:
y_test_pred

In [43]:
y_test_pred[0].value_counts()

0    9913
1     384
Name: 0, dtype: int64

In [0]:
y_test_pred['id'] = test_features['id']

In [0]:
y_test_pred = y_test_pred.rename(columns={0: 'y'})

In [0]:
from google.colab import files

y_test_pred.to_csv('test_labels.csv',index=False)
files.download('test_labels.csv')

In [41]:
y_train.head()

5816     0
12650    0
2992     0
7194     0
2082     0
Name: y, dtype: int64

In [42]:
X_train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
5816,34,admin.,divorced,high.school,no,no,no,cellular,apr,tue,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1
12650,36,blue-collar,single,university.degree,unknown,no,no,telephone,jun,mon,2,999,0,nonexistent,1.4,94.465,-41.8,4.960,5228.1
2992,39,technician,married,professional.course,no,yes,no,cellular,aug,thu,1,999,0,nonexistent,1.4,93.444,-36.1,4.968,5228.1
7194,35,blue-collar,married,basic.9y,unknown,no,no,telephone,may,thu,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0
2082,48,management,divorced,university.degree,no,no,no,telephone,may,wed,2,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0


In [53]:
sample_submission.head()
#sample_submission.shape

,id,y
0,12668,0.5
1,44327,0.5
2,26675,0.5
3,11771,0.5
4,38173,0.5


In [61]:
y_test_pred.shape

(10297, 2)

In [62]:
y_test_pred.head()

,y,id
0,0,12668
1,0,44327
2,0,26675
3,0,11771
4,0,38173
